# (WI4650) CodeLab 7 - QML
### Problem: QML for ordinary differential equations

In this Codelab you will learn how to implement a QML for the ordinary differential equation (ODE) $\frac{df}{dx}=4x^3+x^2-2x-\frac{1}{2}$ with $f(0)=1$ using Qadence.

To get started, please do the following:
1. Install [Qadence](https://pasqal-io.github.io/qadence/latest/) and some additional Python packages: `pip install jupyterlab qadence tqdm pandas seaborn`
2. Start JubyterLab: `jupyter lab`

### A working QML example

[Qadence](https://pasqal-io.github.io/qadence/latest/) is yet another quantum programming package that is particularly useful for developing QML applications.

In [ ]:
# General imports
from time import perf_counter
from typing import Callable

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from numpy.random import uniform
from qadence import (
    QNN,
    BasisSet,
    QuantumCircuit,
    chain,
    feature_map,
    hea,
    ising_hamiltonian,
)
from torch import linspace, manual_seed, ones_like, optim, tensor, zeros_like
from torch.autograd import grad
from tqdm import tqdm

In [ ]:
# Setup the function that we're trying to learn
def ode(inputs: tensor) -> tensor:
    return 4 * inputs**3 + inputs**2 - 2 * inputs - 0.5


def analytical(inputs: tensor) -> tensor:
    return inputs**4 + (1 / 3) * inputs**3 - inputs**2 - (1 / 2) * inputs + 1

Let us define some global parameters. You can later change these parameters to study how less or more qubits or a deeper quantum network influences the QML's performance.

In [ ]:
N_QUBITS, DEPTH, LEARNING_RATE, N_POINTS = 4, 3, 0.01, 20

An essential part of a QML is the **ansatz**, that is, a parametric circuit whose parameters will be trained. 
Let us use the **hardware-efficient ansatz** ([`hea`](https://pasqal-io.github.io/qadence/latest/api/constructors/#qadence.constructors.hea.hea)).

In [ ]:
ansatz = hea(n_qubits=N_QUBITS, depth=DEPTH)

In [ ]:
display(ansatz)

The other important part is the **feature map**, which encodes the input of our quantum network into the quantum circuit. Here, we use the **Chebyshev feature map**.

In [ ]:
fm = feature_map(n_qubits=N_QUBITS, param="x", fm_type=BasisSet.CHEBYSHEV)

Next, we need to define a **cost function**. Don't confuse this with the loss function that encodes our ODE. Let us choose the **transverse-field Ising Hamiltonian**.

In [ ]:
obs = ising_hamiltonian(n_qubits=N_QUBITS)

Now we are ready to build the *quantum circuit* and the **QNN model**

In [ ]:
circuit = QuantumCircuit(N_QUBITS, chain(fm, ansatz))
model = QNN(circuit=circuit, observable=obs, inputs=["x"])

Next, we need to implement the **loss function**. In our case, we implement the MSE loss function for the ODE $\frac{df}{dx}=4x^3+x^2-2x-1/2$ with $f(0)=1$

In [ ]:
def loss_fn(inputs: tensor, outputs: tensor, ode: Callable[[tensor], tensor]) -> tensor:
    dfdx = grad(inputs=inputs, outputs=outputs.sum(), create_graph=True)[0]
    ode_loss = dfdx - ode(inputs)
    boundary_loss = model(zeros_like(inputs)) - ones_like(inputs)
    return ode_loss.pow(2).mean() + boundary_loss.pow(2).mean()

Let us train the QNN for 1000 epochs with randomly samples collocation points between (-1.0, 1.0)

In [ ]:
def benchmark(
    domain_interval,
    loss_fn,
    ode,
    num_epochs,
    optimizer,
    model,
    analytical_sol,
):
    result_dict = {"optimizer": [], "loss": [], "accuracy": [], "time": [], "epoch": []}
    sample_points = (
        linspace(domain_interval[0], domain_interval[1], steps=100)
        .reshape(-1, 1)
        .detach()
    )
    for epoch in tqdm(range(num_epochs)):
        start_time = perf_counter()
        optimizer.zero_grad()

        # the collocation points are sampled randomly
        cp = tensor(
            uniform(
                low=domain_interval[0], high=domain_interval[1], size=(N_POINTS, 1)
            ),
            requires_grad=True,
        ).float()

        loss = loss_fn(inputs=cp, outputs=model(cp), ode=ode)
        end_time = perf_counter()
        loss.backward()
        optimizer.step()

        analytic_sol = analytical_sol(sample_points)
        dqc_sol = model(sample_points).detach().numpy()
        result_dict["optimizer"].append(optimizer.__class__.__name__)
        result_dict["loss"].append(loss.item())
        result_dict["time"].append(end_time - start_time)
        result_dict["epoch"].append(epoch + 1)
        result_dict["accuracy"].append(
            (np.square(dqc_sol.flatten() - analytic_sol.flatten().numpy())).mean(axis=0)
        )

    return result_dict

In [ ]:
# Define some paramters
interval = (-0.99, 0.99)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
num_epochs = 1000

In [ ]:
result_dict = benchmark(
    interval, loss_fn, ode, num_epochs, optimizer, model, analytical
)

In [ ]:
df = pd.DataFrame.from_dict(result_dict)
df

Now, let us evaluate the solution predicted by the QNN. We start by visually comparing the analytical solution and the DQC model.

In [ ]:
sample_points = linspace(interval[0], interval[1], steps=100).reshape(-1, 1)
dqc_sol = model(sample_points).detach().numpy()
analytic_sol = analytical(sample_points)
x_data = sample_points.detach().numpy()

In [ ]:
sns.set_theme()

plt.figure(figsize=(4, 4))
plt.plot(x_data, analytic_sol.flatten(), color="gray", label="Exact solution")
plt.plot(x_data, dqc_sol.flatten(), color="orange", label="DQC solution")
plt.xlabel(r"$x$")
plt.ylabel(r"$\frac{df}{dx}$")
plt.title(r"$\frac{df}{dx}=4x^3+x^2-2x-\frac{1}{2}$ comparison")
plt.legend()
plt.show()

In [ ]:
f, ax = plt.subplots()
ax.set(yscale="log")

sns.lineplot(data=df, x="epoch", y="accuracy", ax=ax, label="accuracy")
sns.lineplot(data=df, x="epoch", y="loss", ax=ax, label="loss")

## Assignments

Similar to the VQE assignment, your first task is to experiment with different parameters. You can use the code in the cells below to concatenate the results from multiple runs of the `benchmark` function, which should make it easier for you to visualize and analyse the data.

In [ ]:
# Set up and train another QNN from scratch
def train_model(
    optimizer_class, n_qubits, depth, num_epochs, domain_interval, learning_rate
):
    ansatz = hea(n_qubits=n_qubits, depth=depth)
    fm = feature_map(n_qubits=n_qubits, param="x", fm_type=BasisSet.CHEBYSHEV)
    obs = ising_hamiltonian(n_qubits=n_qubits)
    circuit = QuantumCircuit(n_qubits, chain(fm, ansatz))
    model = QNN(circuit=circuit, observable=obs, inputs=["x"])

    optimizer = optimizer_class(model.parameters(), lr=learning_rate)
    results = benchmark(
        domain_interval, loss_fn, ode, num_epochs, optimizer, model, analytical
    )
    return results

In [ ]:
N_QUBITS, DEPTH, LEARNING_RATE, NUM_EPOCHS, DOMAIN_INTERVAL = (
    4,
    3,
    0.01,
    500,
    (-0.99, 0.99),
)

dfs = []

for optimizer in [optim.Adam, optim.SGD]:
    result_dict = train_model(
        optimizer,
        N_QUBITS,
        DEPTH,
        NUM_EPOCHS,
        DOMAIN_INTERVAL,
        LEARNING_RATE,
    )
    dfs.append(pd.DataFrame.from_dict(result_dict))

In [ ]:
cdf = pd.concat(dfs)

plt.figure(figsize=(10, 7))
ax0 = plt.subplot(121)
ax0.set(yscale="log")


ax1 = plt.subplot(122, sharex=ax0)
ax1.set(yscale="log")

sns.lineplot(data=cdf, x="epoch", y="accuracy", ax=ax0, hue="optimizer")
sns.lineplot(data=cdf, x="epoch", y="loss", ax=ax1, hue="optimizer")

### Assignment 1 - Number of qubits

Tweak the QNN implementation by varying the number of qubits. Keep all other parameters unchanged. What effects does this have on the accuracy and cost of your model?

In [ ]:
# Your code here...

Your analysis here...

### Assignment 2 - Number of qubits

Select a number of qubits that performs well and takes a reasonable time to train. Now tweak the QNN implementation by varying the depth of the ansatz. Keep all other parameters unchanged. What effects does this have on the accuracy and cost of your model?

In [ ]:
# Your code here...

Your analysis here...

### Assignment 3 - Feature maps

Again select a combination of parameters that perfoms well and can be trained reasonably quickly. Now experiment with modifying the QNN implementation by changing the feature map (for instance [`BasisSet.FOURIER`](https://pasqal-io.github.io/qadence/v1.5.1/qadence/types/#qadence.types.BasisSet.FOURIER)). Keep all other parameters unchanged. What effects does this have on the accuracy and cost of your model?

In [ ]:
# Your code here...

Your analysis here...

### Assignment 4 - Optimizers

Finally, repeat the previous assignment by experiment with other [PyTorch optimizers](https://pytorch.org/docs/stable/optim.html), e.g., `optim.LBFGS`, `optim.RMSprop`, or `optim.SGD`. How does this change the training process?

In [ ]:
# Your code here...

Your analysis here...

### Assignment 5 - A different equation

 Your final task is to use the experience you gained in the previous assignments to solve a different equation from scratch. Use a sensible combination of parameters which you found in the previous 4 assignment. Implement all necessary components to solve a different ODE, namely, $\frac{df}{dy}=\cos(x)$ with $y(0)=0$. How does your parameter choice perform for this equation? Which choice do you think has the biggest impact?

In [ ]:
# Your code here...

## Feedback

We are looking to improve these notebooks and would greatly appreciate your thoughts and feedback.

What was your impression of this assignment? Do you have any suggestions about how we could improve this notebook for next year? Are there any changes you would make to the assignments that would make them more interesting or more instructive? Are there any assignments that you found too difficult, boring, or not instructive?

Any and all feedback is welcome!

Your feedback here!